In [6]:
from pathlib import Path
import pyrosetta
import pyrosetta_help as ph
from types import ModuleType
from IPython.display import display, HTML

from collections import Counter
prc: ModuleType = pyrosetta.rosetta.core
prp: ModuleType = pyrosetta.rosetta.protocols
pru: ModuleType = pyrosetta.rosetta.utility
pro: ModuleType = pyrosetta.rosetta.basic.options
prcc: ModuleType = pyrosetta.rosetta.core.conformation
pr_scoring: ModuleType = pyrosetta.rosetta.core.scoring
pr_rs: ModuleType = pyrosetta.rosetta.core.select.residue_selector


logger = ph.configure_logger()
pyrosetta.distributed.maybe_init(extra_options=ph.make_option_string(no_optH=False,
                                                                     ex1=None,
                                                                     ex2=None,
                                                                     # mute='all',
                                                                     ignore_unrecognized_res=True,
                                                                     load_PDB_components=False,
                                                                     ignore_waters=True)
                                 )

pose: pyrosetta.Pose = pyrosetta.pose_from_file('collage.pdb')

In [35]:
first_resi = pose.sequence().find('GQQSGAIYVG')

In [36]:
blue = ph.Blueprinter.from_pose(pose)

for i in range(-10, -4+1):
    blue.pick_native(i + first_resi)
blue.set('mut2.blu')

In [37]:
pr_options.set_boolean_option('remodel:quick_and_dirty', False)
pr_options.set_string_option('remodel:generic_aa', 'G')

rm = prp.forge.remodel.RemodelMover()
rm.register_options()
rm.dr_cycles(15) # default 3
rm.max_linear_chainbreak(0.2) # default 0.07
rm.redesign_loop_neighborhood(False)
rm.apply(pose)

AA for build: GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG


In [43]:
pose.sequence()

'GDRVADVIESSIGDSVSRALTQALPAPTGQNTQVSSHRLDTGKVPALQAAEIGASSNASDESMIETRCVLNSHSTAETTLDSFFSRAGLVGEIDLPLEGTTNPNGYCNWDIDITGYAQMRRKVELFTYMRFDAEFTFVACTPTGEVVPQLLQYMFVPPGAPKPDSRESLAWQTATNPSVFVKLSDPPAQVSVPFMSPASAYQWFYDGYPTFGEHKQEKDLEYGACPNNMMGTFSVRTVGTSKSKYPLVVRIYMRMKHVRAWIPRPMRNQNYLFKANPNYAGNSIKPTGTSRTAITTLGKFGQQSGAIYVGNFRVVNRHLATHNDWANLVWEDSSRDLLVSSTTAQGCDTIARCNCQTGVYYCNSRRKHYPVSFSKPSLIYVEASEYYPARYQSHLMLAQGHSEPGDAGGILRCQHGVVGIVSTGGNGLVGFADVRDLLWLDEEAMEQ'

In [39]:
pose.dump_pdb('tweaked.pdb')

True

In [46]:
pose.residue(407).name3()

'ALA'

In [47]:
MutateResidue = prp.simple_moves.MutateResidue
MutateResidue(target=407, new_res='CYS').apply(pose)

In [52]:
scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
#relax.constrain_relax_to_start_coords(True)
movemap = pyrosetta.MoveMap()
ri_sele = pr_rs.ResidueIndexSelector()
first_resi = pose.sequence().find('GQQSGAIYVG')
for i in range(-15, -4+1):
    ri_sele.append_index(i+first_resi)
wobbled: pru.vector1_bool = ri_sele.apply(pose)
movemap.set_bb(wobbled)
movemap.set_chi(wobbled)
relax.set_movemap(movemap)
relax.apply(pose)

In [53]:
pose.dump_pdb('tweaked.relaxStep1.pdb')

True

In [54]:
pro: ModuleType = pyrosetta.rosetta.basic.options

scorefxn: pr_scoring.ScoreFunction = pyrosetta.get_fa_scorefxn()
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 5)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
pyrosetta.rosetta.basic.options.set_boolean_option('relax:constrain_relax_to_start_coords', True)
pyrosetta.rosetta.basic.options.set_boolean_option('relax:coord_constrain_sidechains', True)
pyrosetta.rosetta.protocols.relax.FastRelax.register_options()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
relax.constrain_relax_to_start_coords(True)
movemap = pyrosetta.MoveMap()
movemap.set_bb(False)
movemap.set_chi(True)
relax.set_movemap(movemap)
relax.apply(pose)
pose.dump_pdb('tweaked.relaxStep2.pdb')

True

In [ ]:
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 2)
movemap = pyrosetta.MoveMap()
movemap.set_bb(True)
movemap.set_chi(True)
relax.set_movemap(movemap)
relax.apply(pose)
pose.dump_pdb('tweaked.relaxStep3.pdb')

IOStream.flush timed out


In [ ]:
scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 0)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
relax.apply(pose)
pose.dump_pdb('tweaked.relaxed.pdb')

In [ ]:
atoms = (ph.get_AtomID_from_pymol_line(pose, '//A/LYS`299/NZ '),
         ph.get_AtomID_from_pymol_line(pose, '//A/GLU`385/OE2 ')
        )
con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=2.1, sd_in=0.5))
pose.add_constraint(con)

atoms = (ph.get_AtomID_from_pymol_line(pose, '//A/ARG`291/CZ '),
         ph.get_AtomID_from_pymol_line(pose, '//A/GLU`382/OE2 ')
        )
con = AtomPairConstraint(*atoms, HarmonicFunc(x0_in=3.1, sd_in=0.5))
pose.add_constraint(con)

scorefxn.set_weight(pr_scoring.ScoreType.coordinate_constraint, 0)
scorefxn.set_weight(pr_scoring.ScoreType.angle_constraint, 5)
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 5)
relax.apply(pose)
pose.dump_pdb('tweaked.relaxed2.pdb')